In [1]:
pip install ace_tools_open


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\nithi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install imblearn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\nithi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\nithi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
# 1. Imports
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score


In [5]:

# 2. Load Datasets
df = pd.read_csv(r"C:\Users\nithi\OneDrive - St. Clair College\322-capstone-project\benefits-and-cost-sharing-puf\cleaned_benefits_data.csv")
group_ref = pd.read_csv(r"C:\Users\nithi\Downloads\ML-model\BENEFIT_GROUPED.csv")


C:\Users\nithi\AppData\Local\Temp\ipykernel_18196\960638996.py:2: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\nithi\OneDrive - St. Clair College\322-capstone-project\benefits-and-cost-sharing-puf\cleaned_benefits_data.csv")


In [6]:
df.columns

Index(['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'ImportDate',
       'StandardComponentId', 'PlanId', 'BenefitName', 'CopayOutofNet',
       'CoinsOutofNet', 'IsEHB', 'IsCovered', 'QuantLimitOnSvc', 'LimitQty',
       'LimitUnit', 'Explanation', 'EHBVarReason', 'IsExclFromInnMOOP',
       'IsExclFromOonMOOP', 'CoinsInnTier1', 'CoinsInnTier2', 'CopayInnAmount',
       'CopayInnText', 'CopayInnAmount_2', 'CopayInnText_2', 'MOOP',
       'MOOP_Rank', 'Coverage_Rank', 'Cost_Rank'],
      dtype='object')

In [7]:
# 4. Drop Irrelevant Columns
irrelevant_cols = ["BusinessYear", "IssuerId", "StandardComponentId", "PlanId", "SourceName", "ImportDate",'CopayInnText_2', 'MOOP_Rank','Coverage_Rank', 'Cost_Rank', 'BenefitGroupID','LimitQty', 'LimitUnit', 'Explanation',
       'EHBVarReason','QuantLimitOnSvc','CopayInnText']
df.drop(columns=[col for col in irrelevant_cols if col in df.columns], inplace=True, errors='ignore')


In [8]:
df.columns

Index(['StateCode', 'BenefitName', 'CopayOutofNet', 'CoinsOutofNet', 'IsEHB',
       'IsCovered', 'IsExclFromInnMOOP', 'IsExclFromOonMOOP', 'CoinsInnTier1',
       'CoinsInnTier2', 'CopayInnAmount', 'CopayInnAmount_2', 'MOOP'],
      dtype='object')

In [9]:

# 3. Merge Datasets on Cleaned 'BenefitName'
df["BenefitName"] = df["BenefitName"].astype(str).str.strip().str.title()
group_ref["BenefitName"] = group_ref["BenefitName"].astype(str).str.strip().str.title()
df = pd.merge(df, group_ref, how="left", on="BenefitName")

# Drop rows with missing target group info
df.dropna(subset=["BenefitGroupID"], inplace=True)



In [10]:
df.head()

,StateCode,BenefitName,CopayOutofNet,CoinsOutofNet,IsEHB,IsCovered,IsExclFromInnMOOP,IsExclFromOonMOOP,CoinsInnTier1,CoinsInnTier2,CopayInnAmount,CopayInnAmount_2,MOOP,BenefitGroupID
0,AK,Routine Dental Services (Adult),Not Applicable,20.00%,No,Covered,Yes,Yes,20.0,10.783632,0.0,0.0,20.0,86.0
1,AK,Dental Check-Up For Children,Not Applicable,15.00%,Yes,Covered,No,No,15.0,10.783632,0.0,0.0,15.0,16.0
2,AK,Basic Dental Care - Child,Not Applicable,60.00%,Yes,Covered,No,No,60.0,10.783632,0.0,0.0,60.0,7.0
3,AK,Orthodontia - Child,Not Applicable,70.00%,Yes,Covered,No,No,70.0,10.783632,0.0,0.0,70.0,61.0
4,AK,Major Dental Care - Child,Not Applicable,70.00%,Yes,Covered,No,No,70.0,10.783632,0.0,0.0,70.0,47.0


In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load cleaned dataset
df = pd.read_csv(r"C:\Users\nithi\OneDrive - St. Clair College\322-capstone-project\benefits-and-cost-sharing-puf\cleaned_benefits_data.csv", low_memory=False, nrows=30000)

# Normalize BenefitName and merge group info
df["BenefitName"] = df["BenefitName"].astype(str).str.strip().str.title()
group_ref = pd.read_csv("BENEFIT_GROUPED.csv")
group_ref["BenefitName"] = group_ref["BenefitName"].astype(str).str.strip().str.title()
df = pd.merge(df, group_ref, how="left", on="BenefitName")
df.dropna(subset=["BenefitGroupID"], inplace=True)

# ---------------------------------------
# STEP 1: Numeric conversion + clean up
# ---------------------------------------
cost_cols = [
    "CopayInnAmount", "CopayInnAmount_2", "CopayOutofNet",
    "CoinsInnTier1", "CoinsInnTier2", "CoinsOutofNet", "MOOP"
]

# Replace "Not Applicable" with 0 and clean non-numeric
for col in cost_cols:
    df[col] = df[col].astype(str).str.replace("Not Applicable", "0", case=False)
    df[col] = df[col].str.replace(r"[^\d.]", "", regex=True)
    df[col] = pd.to_numeric(df[col], errors="coerce")

# ---------------------------------------
# STEP 2: Add binary flags for zero-values
# ---------------------------------------
for col in cost_cols:
    df[f"{col}_was_zero"] = (df[col] == 0).astype(int)

# ---------------------------------------
# STEP 3: Clip outliers at 99th percentile
# ---------------------------------------
for col in cost_cols:
    upper = df[col].quantile(0.99)
    df[col] = np.clip(df[col], 0, upper)

# ---------------------------------------
# STEP 4: Impute missing values with median
# ---------------------------------------
df[cost_cols] = df[cost_cols].fillna(df[cost_cols].median())

# ---------------------------------------
# STEP 5: Encode boolean fields
# ---------------------------------------
bool_map = {"Yes": 1, "No": 0, "Covered": 1, "Not Covered": 0}
for col in ["IsEHB", "IsCovered", "IsExclFromInnMOOP", "IsExclFromOonMOOP"]:
    df[col] = df[col].map(bool_map).fillna(0).astype(int)



In [12]:

# ---------------------------------------
# STEP 6: Encode target labels
# ---------------------------------------
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["GroupLabel"] = le.fit_transform(df["BenefitGroupID"])



In [13]:

# ---------------------------------------
# STEP 7: Scale features and combine
# ---------------------------------------
scaler = StandardScaler()
scaled_costs = scaler.fit_transform(df[cost_cols])
scaled_costs_df = pd.DataFrame(scaled_costs, columns=cost_cols)


In [14]:

# Combine with zero flags and booleans
flag_cols = [f"{col}_was_zero" for col in cost_cols]
X = pd.concat([
    scaled_costs_df.reset_index(drop=True),
    df[flag_cols].reset_index(drop=True),
    df[["IsEHB", "IsCovered", "IsExclFromInnMOOP", "IsExclFromOonMOOP"]].reset_index(drop=True)
], axis=1)

y = df["GroupLabel"]


In [15]:
# 1. Imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE 

# 2. Initialize and Train Random Forest Model
rf_model = RandomForestClassifier(
    n_estimators=200,class_weight='balanced',
    max_depth=20,
    random_state=42,
    n_jobs=-1
)
sm = SMOTE(random_state=42, k_neighbors=1)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)
rf_model.fit(X_resampled, y_resampled)

# 3. Predict on Test Set
y_pred = rf_model.predict(X_test)

# 4. Evaluate the Model
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

print("✅ Random Forest Classifier Performance")
print("Accuracy:", round(accuracy * 100, 2), "%")
print("F1 Score (macro):", round(f1, 4))
print("\nClassification Report:\n", classification_report(y_test, y_pred))



NameError: name 'X_train' is not defined

In [ ]:
print(y.value_counts())


GroupLabel
81    512
13    511
6     511
56    511
43    511
     ... 
53      6
62      6
63      6
64      6
32      6
Name: count, Length: 106, dtype: int64


# RandomForestClassifier

In [ ]:
# 1. Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 2. Load data
df = pd.read_csv(r"C:\Users\nithi\OneDrive - St. Clair College\322-capstone-project\benefits-and-cost-sharing-puf\cleaned_benefits_data.csv")
group_map = pd.read_csv(r"C:\Users\nithi\Downloads\ML-model\benefit_supercategories_cleaned.csv")

# 3. Merge SuperCategoryID
df["BenefitName"] = df["BenefitName"].astype(str).str.strip().str.title()
group_map["BenefitName"] = group_map["BenefitName"].astype(str).str.strip().str.title()
df = pd.merge(df, group_map[["BenefitName", "SuperCategoryID"]], how="left", on="BenefitName")

# ❌ Remove invalid CategoryIDs
df = df[df["SuperCategoryID"].notna()]
df = df[df["SuperCategoryID"] != -1]
df["SuperCategoryID"] = df["SuperCategoryID"].astype(int)

# 4. Clean cost columns
cost_cols = [
    "CopayInnAmount", "CopayInnAmount_2", "CopayOutofNet",
    "CoinsInnTier1", "CoinsInnTier2", "CoinsOutofNet", "MOOP"
]

for col in cost_cols:
    df[col] = df[col].astype(str).str.replace("Not Applicable", "0", case=False)
    df[col] = df[col].str.replace(r"[^\d.]", "", regex=True)
    df[col] = pd.to_numeric(df[col], errors="coerce")
    df[f"{col}_was_zero"] = (df[col] == 0).astype(int)
    df[col] = np.clip(df[col], 0, df[col].quantile(0.99))

df[cost_cols] = df[cost_cols].fillna(df[cost_cols].median())

# 5. Encode binary features
bool_map = {"Yes": 1, "No": 0, "Covered": 1, "Not Covered": 0}
bool_fields = ["IsEHB", "IsCovered", "IsExclFromInnMOOP", "IsExclFromOonMOOP"]
for col in bool_fields:
    df[col] = df[col].map(bool_map).fillna(0).astype(int)

# 6. Prepare features
binary_flags = [f"{col}_was_zero" for col in cost_cols]
features = cost_cols + binary_flags + bool_fields
X = df[features]
y = df["SuperCategoryID"]

# 7. Scale & split
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# 8. Train Random Forest with best hyperparams
rf_model = RandomForestClassifier(
    n_estimators=150,
    max_depth=20,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)

# 9. Predict & Evaluate
y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

print("✅ Cleaned Random Forest Model with Valid CategoryID")
print("Accuracy:", round(accuracy * 100, 2), "%")
print("F1 Score (macro):", round(f1, 4))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


C:\Users\nithi\AppData\Local\Temp\ipykernel_20008\4153715621.py:10: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\nithi\OneDrive - St. Clair College\322-capstone-project\benefits-and-cost-sharing-puf\cleaned_benefits_data.csv")


✅ Cleaned Random Forest Model with Valid CategoryID
Accuracy: 25.89 %
F1 Score (macro): 0.2477

Classification Report:
               precision    recall  f1-score   support

           0       0.14      0.20      0.16     12596
           1       0.94      0.31      0.46     28349
           2       0.36      0.70      0.48      9566
           3       0.17      0.18      0.18     14599
           4       0.20      0.54      0.29     14599
           5       0.47      0.26      0.33     14981
           6       0.62      0.01      0.03     26289
           7       0.31      0.39      0.34     12276
           8       0.18      0.31      0.23     11690
           9       0.20      0.23      0.22     17761
          10       0.39      0.02      0.05     14599
          11       0.35      0.17      0.23     17519
          12       0.14      0.49      0.22      8759

    accuracy                           0.26    203583
   macro avg       0.35      0.29      0.25    203583
weighted avg  

# XGBoost

In [20]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from xgboost import XGBClassifier
from scipy.sparse import hstack

# Load datasets
df = pd.read_csv(r"C:\Users\nithi\Downloads\ML-model\cleaned_benefits_data.csv", low_memory=False)
group_map = pd.read_csv(r"C:\Users\nithi\Downloads\ML-model\benefit_supercategories_cleaned.csv", low_memory=False)

# Merge SuperCategoryID
df["BenefitName"] = df["BenefitName"].astype(str).str.strip().str.title()
group_map["BenefitName"] = group_map["BenefitName"].astype(str).str.strip().str.title()
df = pd.merge(df, group_map[["BenefitName", "SuperCategoryID"]], on="BenefitName", how="left")
df = df[df["SuperCategoryID"].notna() & (df["SuperCategoryID"] != -1)]
df["SuperCategoryID"] = df["SuperCategoryID"].astype(int)

# Normalize cost-related fields
cost_cols = [
    "CopayInnAmount", "CopayInnAmount_2", "CopayOutofNet",
    "CoinsInnTier1", "CoinsInnTier2", "CoinsOutofNet", "MOOP"
]
for col in cost_cols:
    df[col] = df[col].astype(str).str.replace("Not Applicable", "0", case=False)
    df[col] = df[col].str.replace(r"[^\d.]", "", regex=True)
    df[col] = pd.to_numeric(df[col], errors="coerce")
    df[f"{col}_was_zero"] = (df[col] == 0).astype(int)
    df[col] = np.clip(df[col], 0, df[col].quantile(0.99))
df[cost_cols] = df[cost_cols].fillna(df[cost_cols].median())

# Encode boolean fields
bool_map = {"Yes": 1, "No": 0, "Covered": 1, "Not Covered": 0}
bool_fields = ["IsEHB", "IsCovered", "IsExclFromInnMOOP", "IsExclFromOonMOOP"]
for col in bool_fields:
    df[col] = df[col].map(bool_map).fillna(0).astype(int)

# Prepare numeric features
binary_flags = [f"{col}_was_zero" for col in cost_cols]
numeric_features = cost_cols + binary_flags + bool_fields
X_numeric = df[numeric_features]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_numeric)
y = df["SuperCategoryID"]

# Vectorizer configuration (selecting best based on previous results)
vectorizer = TfidfVectorizer(max_features=50, ngram_range=(1, 2))
benefit_vectors = vectorizer.fit_transform(df["BenefitName"])
X_combined = hstack([benefit_vectors, X_scaled])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y, test_size=0.2, stratify=y, random_state=42
)

# Train final model
model = XGBClassifier(
    objective="multi:softprob",
    num_class=len(np.unique(y)),
    eval_metric="mlogloss",
    use_label_encoder=False,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

# Save model and transformers
joblib.dump(model, "xgb_benefit_classifier.pkl")
joblib.dump(vectorizer, "benefitname_vectorizer.pkl")
joblib.dump(scaler, "numeric_scaler.pkl")

# Evaluate
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="macro")
report = classification_report(y_test, y_pred)

print("✅ Final Model Saved")
print("Accuracy:", round(acc * 100, 2), "%")
print("F1 Score (macro):", round(f1, 4))
print("\nClassification Report:\n", report)


C:\Users\nithi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\training.py:183: UserWarning: [15:59:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Final Model Saved
Accuracy: 89.84 %
F1 Score (macro): 0.8964

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.30      0.46     12596
           1       1.00      1.00      1.00     28349
           2       0.97      0.94      0.95      9566
           3       1.00      1.00      1.00     14599
           4       0.96      0.94      0.95     14599
           5       0.96      0.98      0.97     14981
           6       0.65      0.92      0.76     26289
           7       1.00      1.00      1.00     12276
           8       0.88      0.96      0.92     11690
           9       0.83      0.71      0.76     17761
          10       1.00      1.00      1.00     14599
          11       0.89      0.85      0.87     17519
          12       1.00      1.00      1.00      8759

    accuracy                           0.90    203583
   macro avg       0.93      0.89      0.90    203583
weighted avg       0.91      0.90      0.89   

In [22]:
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import hstack

# Load the model and transformers
model = joblib.load(r"C:\Users\nithi\Downloads\ML-model\xgb_benefit_classifier.pkl")
vectorizer = joblib.load(r"C:\Users\nithi\Downloads\ML-model\benefitname_vectorizer.pkl")
scaler = joblib.load(r"C:\Users\nithi\Downloads\ML-model\numeric_scaler.pkl")

st.set_page_config(page_title="Healthcare Benefit Classifier", layout="wide")
st.title("🩺 Healthcare Benefit Classification Tool")

st.markdown("""
### 🎯 Objective
Automatically categorize a health benefit into one of the key patient-centric categories (e.g., Dental, Emergency, Maternity).
""")

with st.form("benefit_form"):
    st.subheader("Enter Benefit Details")
    benefit_name = st.text_input("Benefit Name", "Basic Dental Care - Adult")

    col1, col2, col3 = st.columns(3)
    with col1:
        copay_inn = st.number_input("Copay In-Network ($)", value=0.0)
        coins_inn = st.number_input("Coinsurance In-Network (%)", value=0.0)
    with col2:
        copay_out = st.number_input("Copay Out-of-Network ($)", value=0.0)
        coins_out = st.number_input("Coinsurance Out-of-Network (%)", value=0.0)
    with col3:
        moop = st.number_input("Max Out-of-Pocket ($)", value=0.0)

    is_ehb = st.selectbox("Is EHB?", ["Yes", "No"])
    is_covered = st.selectbox("Is Covered?", ["Covered", "Not Covered"])
    is_excl_inn = st.selectbox("Excluded from In-Network MOOP?", ["Yes", "No"])
    is_excl_oon = st.selectbox("Excluded from Out-of-Network MOOP?", ["Yes", "No"])

    submitted = st.form_submit_button("Classify Benefit")

if submitted:
    # Preprocess features
    bool_map = {"Yes": 1, "No": 0, "Covered": 1, "Not Covered": 0}
    numeric_data = pd.DataFrame([{
        "CopayInnAmount": copay_inn,
        "CoinsInnTier1": coins_inn,
        "CopayOutofNet": copay_out,
        "CoinsOutofNet": coins_out,
        "MOOP": moop,
        "CopayInnAmount_was_zero": int(copay_inn == 0),
        "CoinsInnTier1_was_zero": int(coins_inn == 0),
        "CopayOutofNet_was_zero": int(copay_out == 0),
        "CoinsOutofNet_was_zero": int(coins_out == 0),
        "MOOP_was_zero": int(moop == 0),
        "IsEHB": bool_map[is_ehb],
        "IsCovered": bool_map[is_covered],
        "IsExclFromInnMOOP": bool_map[is_excl_inn],
        "IsExclFromOonMOOP": bool_map[is_excl_oon]
    }])

    X_scaled = scaler.transform(numeric_data)
    X_text = vectorizer.transform([benefit_name])
    X_final = hstack([X_text, X_scaled])

    prediction = model.predict(X_final)[0]
    categories = [
        "Dental", "Mental Health", "Emergency", "Maternity",
        "Primary Care", "Pharmacy", "Hospital", "Other",
        "Vision", "Specialty Care", "Behavioral Health", "Preventive Care"
    ]

    st.success(f"✅ Predicted Category: **{categories[prediction]}**")

    st.markdown("""
    ### 🏥 Why This Helps Patients
    - **Simplifies Plan Comparison**: Grouping helps patients compare benefits side-by-side.
    - **Feeds Recommendation Engines**: Used in matching plans with patient conditions.
    - **Aids Regulators**: Classifies benefits to verify EHB compliance.
    - **Saves Time**: Reduces the manual work of benefit categorization.
    """)

    st.markdown("---")
    st.caption("Built with ❤️ using Streamlit")

2025-04-06 16:02:49.032 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 16:02:49.034 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 16:02:49.128 
  command:

    streamlit run C:\Users\nithi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-06 16:02:49.129 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 16:02:49.131 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 16:02:49.132 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 1